# 1. Load the Drivers dataset

In [2]:
import pandas as pd

drivers = pd.read_csv('./data/drivers.csv')
circuits = pd.read_csv('./data/circuits.csv')
race_info = pd.read_csv('./data/races.csv')
race_results = pd.read_csv('./data/results.csv')
drivers.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


# 2. Find the Driver that has the longest surname

In [3]:
surname_length = drivers['surname'].apply(lambda x: len(x))

# Without the iloc, we don't take the row, but the overall result. So in the print, it gives ids and dtypes.
longest_surname_driver = drivers[drivers['surname'].apply(lambda x: len(x)) == surname_length.max()].iloc[0]
print(f"The driver with the longest surname is: {longest_surname_driver['forename'] + " " + longest_surname_driver['surname']}")


The driver with the longest surname is: Robin Montgomerie-Charrington


# 3. Find the Oldest Driver

In [4]:
oldest_driver = drivers[drivers['dob'] == drivers['dob'].min()].iloc[0]
print(f"The Oldest Driver ever is: {oldest_driver['forename']+" "+oldest_driver['surname']}")

The Oldest Driver ever is: Philippe Étancelin


# 4. Give the top 10 Drivers with the most races.

In [5]:
import numpy as np

driver_race_count = race_results['driverId'].value_counts(ascending=False).head(10)
top_10_drivers = drivers \
    .join(driver_race_count, on='driverId', how='inner')[['forename', 'surname', 'count']] \
    .sort_values('count',ascending=False) \
    .set_index(np.arange(1,11)) \
    .rename(columns={'forename': "First Name", 'surname': 'Surname', 'count':'Race Starts'})
    
top_10_drivers



,First Name,Surname,Race Starts
1,Fernando,Alonso,404
2,Lewis,Hamilton,356
3,Kimi,Räikkönen,352
4,Rubens,Barrichello,326
5,Jenson,Button,309
6,Michael,Schumacher,308
7,Sebastian,Vettel,300
8,Sergio,Pérez,283
9,Felipe,Massa,271
10,Riccardo,Patrese,257


# 5. Top 10 Currently Youngest Driver's

In [6]:
def age(dob: str):
    year_of_birth = int(dob.split('-')[0])
    return 2025 - year_of_birth

sorted_youngest = drivers.sort_values(by='dob', ascending=False)[['forename', 'surname']]
sorted_youngest['age'] = drivers['dob'].apply(age)
sorted_youngest.head(10).set_index(np.arange(1,11))


,forename,surname,age
1,Oliver,Bearman,20
2,Franco,Colapinto,22
3,Jack,Doohan,22
4,Liam,Lawson,23
5,Oscar,Piastri,24
6,Logan,Sargeant,25
7,Yuki,Tsunoda,25
8,Lando,Norris,26
9,Guanyu,Zhou,26
10,Mick,Schumacher,26


# 6. Top 10 Drivers with most wins ever.

In [7]:
driver_pos = race_results[['driverId','position']]

driver_wins = driver_pos[race_results['position'] == '1'] \
    .groupby('driverId')\
    .count()\
    .rename(columns={'position':'Wins'})\
    .reset_index()\
    .merge(drivers, on='driverId', how='inner')[['forename','surname','Wins']]\
    .sort_values(by='Wins', ascending=False)\
    .head(10)\
    .set_index(np.arange(1,11))\
    .rename(columns={'forename': 'First Name', 'surname':'LastName'})
    
driver_wins


,First Name,LastName,Wins
1,Lewis,Hamilton,105
2,Michael,Schumacher,91
3,Max,Verstappen,63
4,Sebastian,Vettel,53
5,Alain,Prost,51
6,Ayrton,Senna,41
7,Fernando,Alonso,32
8,Nigel,Mansell,31
9,Jackie,Stewart,27
10,Niki,Lauda,25


# 7. Top 10 Drivers with most points.

In [8]:
points_sum = (
    race_results
        .groupby('driverId', as_index=False)['points']
        .sum(numeric_only=True)
        .rename(columns={'points':'Points'})
) 
        
"""
    Use nlargest instead of sorting and then head.
    Also .loc for selecting all rows and then specified cols.
"""

top_10_points = ( 
    drivers
        .merge(points_sum, on='driverId', how='inner') 
        .nlargest(10, 'Points')
        .loc[:, ['forename', 'surname', 'Points']]
        .set_index(np.arange(1,11))
        .rename(columns={'forename' : 'First Name', 'surname' : "Surname"})
        .set_index(pd.RangeIndex(1, 11, name='Rank'))
)

top_10_points

,First Name,Surname,Points
Rank,,,
1,Lewis,Hamilton,4820.5
2,Sebastian,Vettel,3098.0
3,Max,Verstappen,2912.5
4,Fernando,Alonso,2329.0
5,Kimi,Räikkönen,1873.0
6,Valtteri,Bottas,1788.0
7,Nico,Rosberg,1594.5
8,Sergio,Pérez,1585.0
9,Michael,Schumacher,1566.0


# 8. Top 5 Drivers with most Podium finishes

In [9]:
results = race_results.copy()

results['position'] = (
    results['position']
        .fillna(20)
        .replace(to_replace='\\N', value=99)
        .astype(np.int16))

podium_finishes = (
    results[['driverId', 'position']][results['position'] < 4]
        .groupby('driverId', as_index=False)
        .agg(Podiums = ('position', 'count'))
        .merge(drivers[['driverId', 'forename', 'surname']],on='driverId', how='inner')[['forename', 'surname', 'Podiums']]
        .nlargest(10, 'Podiums')
        .set_index(pd.RangeIndex(1,11, name='Rank'))
        .rename(columns={'forename': 'FirstName', 'surname':'Surname'})
        
)

podium_finishes

,FirstName,Surname,Podiums
Rank,,,
1,Lewis,Hamilton,202
2,Michael,Schumacher,155
3,Sebastian,Vettel,122
4,Max,Verstappen,112
5,Fernando,Alonso,106
6,Alain,Prost,106
7,Kimi,Räikkönen,103
8,Ayrton,Senna,80
9,Rubens,Barrichello,68


# 9. Top 10 Youngest F1 Race Starters

In [10]:
def calculate_age(x):
    dob, date = x
    days_of_first_win = (date - dob).days
    return round(days_of_first_win / 365, ndigits=2)


   

df = drivers.copy()
df['dob'] = pd.to_datetime(df['dob'])

driver_first_race = (
    race_results[['raceId', 'driverId']]
        .merge(race_info[['raceId', 'year', 'date']], on='raceId')
        .groupby('driverId', as_index=False)
        .agg({'year': 'min', 'date':'min'}) 
)

top_10_youngest_race_starters = (
    df[['driverId', 'forename', 'surname', 'dob']]
        .merge(driver_first_race[['driverId', 'date']], on='driverId') 
)

top_10_youngest_race_starters['date'] = pd.to_datetime(top_10_youngest_race_starters['date'])

top_10_youngest_race_starters['Age at 1st Race'] = (
    top_10_youngest_race_starters[['dob', 'date']]
        .apply(lambda x: calculate_age(x), axis=1)
)

top_10_youngest_race_starters \
    .nsmallest(10, 'Age at 1st Race') \
    .set_index(pd.RangeIndex(start=1,stop=11, name='Rank'))[['forename', 'surname', 'Age at 1st Race']] \
    .rename(columns={'forename': 'First Name', 'surname': 'Surname'})


,First Name,Surname,Age at 1st Race
Rank,,,
1,Max,Verstappen,17.47
2,Thomas,Monarch,18.16
3,Lance,Stroll,18.42
4,Oliver,Bearman,18.85
5,Lando,Norris,19.35
6,Jaime,Alguersuari,19.36
7,Mike,Thackwell,19.44
8,Ricardo,Rodríguez,19.58
9,Fernando,Alonso,19.61


# 10. Top 10 Youngest Race Winners

In [11]:
race_info['date'] = pd.to_datetime(race_info['date'])
drivers['dob'] = pd.to_datetime(drivers['dob'])

merged_df = (
    race_results
        .merge(race_info, on='raceId')[['driverId', 'date', 'position']]
)

drivers_first_win = (
    merged_df[merged_df['position'] == '1']
        .groupby('driverId')
        .agg({'date':'min'})
        .merge(drivers[['forename','surname', 'driverId', 'dob']], on='driverId')
)
drivers_first_win['Age at 1st Win'] = drivers_first_win[['dob', 'date']].apply(lambda x : calculate_age(x), axis=1)

top_10_youngest_race_winners = (
    drivers_first_win
        .nsmallest(10, 'Age at 1st Win')[['forename','surname', 'Age at 1st Win']]
        .rename(columns={'forename': 'First Name', 'surname': 'Surname'})
        .set_index(pd.RangeIndex(start=1, stop=11, name='Rank'))
)

top_10_youngest_race_winners


,First Name,Surname,Age at 1st Win
Rank,,,
1,Max,Verstappen,18.64
2,Sebastian,Vettel,21.22
3,Charles,Leclerc,21.89
4,Fernando,Alonso,22.08
5,Troy,Ruttman,22.24
6,Bruce,McLaren,22.30
7,Lewis,Hamilton,22.44
8,Oscar,Piastri,23.31
9,Kimi,Räikkönen,23.45


# 11. Top 10 Drivers that have raced on most tracks

In [ ]:
races_circuits = race_info[['raceId', 'circuitId']]

drivers_raced = (
    race_results[['raceId', 'driverId']]
        .merge(races_circuits, on='raceId')
        .groupby('driverId')['circuitId']
        .nunique('circuitId')
        .reset_index(name="Unique Circuits")
        .merge(drivers[['forename', 'surname', 'driverId']], on='driverId')[['forename','surname','Unique Circuits']]
        .nlargest(10, 'Unique Circuits')
        .set_index(pd.RangeIndex(start=1, stop=11))
        .rename(columns={'forename': 'First Name', 'surname': 'Surname'})
)

drivers_raced

,First Name,Surname,Unique Circuits
1,Lewis,Hamilton,38
2,Fernando,Alonso,37
3,Sebastian,Vettel,37
4,Riccardo,Patrese,37
5,Kimi,Räikkönen,35
6,Graham,Hill,35
7,Sergio,Pérez,35
8,Daniel,Ricciardo,35
9,Michael,Schumacher,34
10,Andrea,de Cesaris,34


# 11. Top 10 Drivers with most Driver Championships